# Question B2 (10 marks)
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [6]:
!pip install pytorch-widedeep --user


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\65976\AppData\Roaming\Python\Python312\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\65976\AppData\Roaming\Python\Python312\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\65976\AppData\Roaming\Python\Python312\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [7]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

1.Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [8]:
import pandas as pd

# Load the data
df = pd.read_csv('hdb_price_prediction.csv')

# Separate data into training and testing sets based on the year
df_train = df[df["year"] <= 2020]
df_test = df[df["year"] >= 2021]

# Print summary information for both sets
print("Training Data Years:", df_train["year"].unique())
print("Testing Data Years:", df_test["year"].unique())

# Check for missing values in both sets
missing_train = df_train.isnull().sum()
missing_test = df_test.isnull().sum()
print("\nMissing Values in Training Data:\n", missing_train[missing_train > 0])
print("\nMissing Values in Testing Data:\n", missing_test[missing_test > 0])

# Display basic statistics for numerical columns in training and testing sets
print("\nTraining Data Statistics:")
display(df_train.describe())

print("\nTesting Data Statistics:")
display(df_test.describe())

# Display the first few rows of the training and testing sets to confirm the structure
print("\nFirst 5 Rows of Training Data:")
display(df_train.head())

print("\nFirst 5 Rows of Testing Data:")
display(df_test.head())


Training Data Years: [2017 2018 2019 2020]
Testing Data Years: [2021 2022 2023]

Missing Values in Training Data:
 Series([], dtype: int64)

Missing Values in Testing Data:
 Series([], dtype: int64)

Training Data Statistics:


,month,year,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,remaining_lease_years,floor_area_sqm,resale_price
count,87370.000000,87370.000000,87370.000000,87370.000000,87370.000000,87370.000000,87370.000000,87370.000000,8.737000e+04
mean,6.887810,2018.554618,0.812743,11.223943,0.017469,0.006544,74.739322,97.723843,4.424959e+05
std,3.326727,1.116347,0.466026,4.175539,0.004832,0.020758,12.945645,24.285644,1.542216e+05
min,1.000000,2017.000000,0.012215,0.677680,0.008403,0.000011,44.833333,31.000000,1.400000e+05
25%,4.000000,2018.000000,0.466695,8.388517,0.016807,0.000086,64.666667,82.000000,3.300000e+05
50%,7.000000,2019.000000,0.711035,12.194836,0.016807,0.000233,74.750000,95.000000,4.100000e+05
75%,10.000000,2020.000000,1.060207,14.271307,0.016807,0.002799,84.083333,113.000000,5.200000e+05
max,12.000000,2020.000000,3.482100,18.639537,0.033613,0.339643,97.750000,249.000000,1.258000e+06



Testing Data Statistics:


,month,year,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,remaining_lease_years,floor_area_sqm,resale_price
count,72183.000000,72183.000000,72183.000000,72183.000000,72183.000000,72183.000000,72183.000000,72183.000000,7.218300e+04
mean,6.036629,2021.824986,0.837095,11.161187,0.017493,0.006510,74.681215,97.086577,5.385027e+05
std,3.354440,0.774247,0.462644,4.105606,0.004753,0.020558,14.713494,23.752890,1.691818e+05
min,1.000000,2021.000000,0.012215,0.677680,0.008403,0.000011,42.500000,31.000000,1.800000e+05
25%,3.000000,2021.000000,0.486837,8.500753,0.016807,0.000090,62.333333,82.000000,4.200000e+05
50%,6.000000,2022.000000,0.743375,12.231067,0.016807,0.000233,74.583333,93.000000,5.120000e+05
75%,9.000000,2022.000000,1.102844,13.957975,0.016807,0.002459,91.000000,112.000000,6.300000e+05
max,12.000000,2023.000000,3.482100,18.639537,0.033613,0.339643,97.416667,243.000000,1.500000e+06



First 5 Rows of Training Data:


,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
0,1,2017,ANG MO KIO,406 ANG MO KIO AVENUE 10,Ang Mo Kio,1.007264,7.006044,0.016807,0.006243,"2 ROOM, Improved",61.333333,44.0,10 TO 12,232000.0
1,1,2017,ANG MO KIO,108 ANG MO KIO AVENUE 4,Ang Mo Kio,1.271389,7.983837,0.016807,0.006243,"3 ROOM, New Generation",60.583333,67.0,01 TO 03,250000.0
2,1,2017,ANG MO KIO,602 ANG MO KIO AVENUE 5,Yio Chu Kang,1.069743,9.090700,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,262000.0
3,1,2017,ANG MO KIO,465 ANG MO KIO AVENUE 10,Ang Mo Kio,0.946890,7.519889,0.016807,0.006243,"3 ROOM, New Generation",62.083333,68.0,04 TO 06,265000.0
4,1,2017,ANG MO KIO,601 ANG MO KIO AVENUE 5,Yio Chu Kang,1.092551,9.130489,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,265000.0



First 5 Rows of Testing Data:


,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
87370,1,2021,ANG MO KIO,170 ANG MO KIO AVENUE 4,Yio Chu Kang,1.276775,8.339960,0.016807,0.002459,"2 ROOM, Improved",64.083333,45.0,01 TO 03,211000.0
87371,1,2021,ANG MO KIO,170 ANG MO KIO AVENUE 4,Yio Chu Kang,1.276775,8.339960,0.016807,0.002459,"2 ROOM, Improved",64.083333,45.0,07 TO 09,225000.0
87372,1,2021,ANG MO KIO,331 ANG MO KIO AVENUE 1,Ang Mo Kio,0.884872,6.981730,0.016807,0.006243,"3 ROOM, New Generation",59.000000,68.0,04 TO 06,260000.0
87373,1,2021,ANG MO KIO,534 ANG MO KIO AVENUE 10,Ang Mo Kio,0.677246,8.333056,0.016807,0.006243,"3 ROOM, New Generation",58.166667,68.0,04 TO 06,265000.0
87374,1,2021,ANG MO KIO,561 ANG MO KIO AVENUE 10,Ang Mo Kio,0.922047,8.009223,0.016807,0.006243,"3 ROOM, New Generation",58.083333,68.0,01 TO 03,265000.0


2.Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [9]:
# YOUR CODE & RESULT HERE

cat_embed_cols=["month", "town", "flat_model_type", "storey_range"]
continuous_cols=["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]


#Use TabPreprocessor to create the deeptabular component using the continuous features and the categorical features. Use this component to transform the training dataset.
tab_preprocessor = TabPreprocessor(
    cat_embed_cols=cat_embed_cols,
    continuous_cols=continuous_cols
    #cols_to_scale=['resale_price'],  # or scale=True or cols_to_scale=continuous_cols
)

#For Training
X_tab = tab_preprocessor.fit_transform(df_train) # x_train
target = df_train['resale_price'].values # y_train

# Create the TabMlp model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
model = TabMlp(
    column_idx=tab_preprocessor.column_idx,  
    cat_embed_input=tab_preprocessor.cat_embed_input, 
    continuous_cols=continuous_cols, 
    #mlp_activation='relu', 
    #mlp_dropout=0.1,
    mlp_hidden_dims=[200, 100] 
     
)

# Create a Trainer for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. 
wide_deep_model = WideDeep(deeptabular=model) # Combine the TabMlp model with any other models you want to use
Trainer_ = Trainer(
    wide_deep_model, 
    objective="root_mean_squared_error", 
    metrics=[R2Score], 
    num_workers=0
)

# Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the num_workers parameter to 0.)
Trainer_.fit(X_tab=X_tab, target=target, n_epochs=100, batch_size=64)


C:\Users\65976\AppData\Roaming\Python\Python312\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:360: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 100: 100%|██████████| 1366/1366 [00:10<00:00, 124.58it/s, loss=4.53e+4, metrics={'r2': 0.9081}]


3.Report the test RMSE and the test R2 value that you obtained.

In [11]:
# Import necessary modules
import math
from sklearn.metrics import r2_score, mean_squared_error

# Transform test data and make predictions
x_test = tab_preprocessor.transform(df_test)
y_test = df_test['resale_price'].values
predictions = Trainer_.predict(X_tab=x_test, batch_size=64)

# Compute RMSE and R2 scores for the model predictions
rmse = math.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

# Display results with improved formatting
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-Squared (R2): {r2:.4f}")


predict: 100%|██████████| 1128/1128 [00:03<00:00, 325.92it/s]

Root Mean Squared Error (RMSE): 102545.49
R-Squared (R2): 0.6326


#### Root Mean Squared Error (RMSE)
Value: 102545.49
Interpretation: On average, the model's predictions deviate from the actual resale prices by approximately 102,545.49 units. While lower values indicate better predictive accuracy, this RMSE suggests that the model's predictions are reasonably close to the actual values, though there may still be room for improvement.
#### R-Squared (R2)
Value: 0.6326
Interpretation: This R2 value means that approximately 63.26% of the variance in the resale prices is explained by the model's input features. An R2 value of 0.6326 indicates a moderately strong fit, suggesting that while the model captures a good portion of the variance, there is still some unexplained variability.
Overall, the model demonstrates a fair performance with these metrics. The R2 score suggests that while the model captures a substantial portion of the variance in resale prices, some improvement may still be possible, particularly in minimizing the RMSE.

### Explanation of the Code:

1. **Data Transformation**:
   - `tab_preprocessor.transform(df_test)`: This line transforms the test data `df_test` using a preprocessor, preparing it for the model. The preprocessor typically scales numeric features and encodes categorical ones, ensuring compatibility with the model.

2. **Extract True Values**:
   - `y_test = df_test['resale_price'].values`: Here, the actual resale prices are extracted from the test dataset. These values are the ground truth for evaluating model predictions.

3. **Model Predictions**:
   - `predictions = Trainer_.predict(...)`: The trained model (`Trainer_`) generates predictions for resale prices based on the transformed test data. The `batch_size=64` argument helps optimize memory usage during prediction by processing the data in smaller chunks.

4. **Evaluation Metrics**:
   - **RMSE (Root Mean Square Error)**:
     - RMSE quantifies the average magnitude of prediction errors, offering insight into how closely the predictions align with the actual values. Lower RMSE values indicate better predictive accuracy.
   - **R2 Score**:
     - The R2 score measures how well the model captures the variability in the target variable (resale price). A score closer to 1 signifies a better fit, indicating that the model explains a large proportion of the variance in resale prices.

5. **Output**:
   - The `print` statements display the RMSE and R2 score to provide a quick assessment of the model's performance on the test set. These metrics are rounded to two decimal places for clarity.


